In [1]:
from itertools import product

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import pysqkit as sq

### Set some matplotlib parameters

### Define some utility functions

In [2]:
transmon = sq.qubits.SimpleTransmon(
    label='transmon', 
    max_freq = 6, 
    anharm = -0.3, 
    dim_hilbert = 5
)

In [3]:
fluxonium = sq.qubits.Fluxonium(
    label='fluxonium',
    joseph_energy=5.900, 
    charge_energy=0.975, 
    induct_energy=0.455,
    flux = 0.5
)
fluxonium.diagonalize_basis(5)

Add the drive (generally you are able to add it after they're coupled. Currently the dimension specified in the Qobj are actually not correct. I'll fix that soon, so basically for now add them before coupling (or fix dimensions manually)

In [4]:
fluxonium.add_drive(
    sq.drives.microwave_drive,
    label='cz_drive',
    pulse_shape=sq.drives.pulse_shapes.gaussian_top
)

In [5]:
system = transmon.couple_to(fluxonium, coupling=sq.couplers.capacitive_coupling, strength=0.2)

you can look at free parameters to be fixed

In [6]:
system['fluxonium'].drives['cz_drive'].free_params

['time', 'rise_time', 'amp', 'freq', 'phase']

if you know phase is always going to be 0, you can go ahead and fix it to the default parameters. Same for some other params

In [7]:
times = np.linspace(0, 60, 10000)
system['fluxonium'].drives['cz_drive'].set_params(phase=0, time=times, rise_time=15)

we see it's been fixed to a value. The other parameters can also be fixed like this or provided when you're evaluating the pulse before doing the Lindblad simulation

In [8]:
drive_params = dict(
    cz_drive=dict(amp = 0.03634, freq = 5.19)
)

hamil = system.hamiltonian(as_qobj=True)
drive_hamils = []
pulses = []

for qubit in system:
    if qubit.is_driven:
        for label, drive in qubit.drives.items():
            drive_hamils.append(drive.hamiltonian(as_qobj=True))
            pulses.append(drive.eval_pulse(**drive_params[label]))

In [9]:
energy, state = system.state('00', as_qobj=True)

In [10]:
result = sq.solvers.integrate(times, state, hamil, drive_hamils, pulses, jump_op=None, solver='mesolve')

In [11]:
pulse  = system['fluxonium'].drives['cz_drive'].eval_pulse(**drive_params['cz_drive'])